In [1]:
import torch
import torchvision
import tqdm
#定义数据集
dataset=torchvision.datasets.MNIST(root='data',train=False,transform=torchvision.transforms.ToTensor(),download=True)

C:\Users\ROG\.conda\envs\pytorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#数据集加载器
batch_size=32#每8条数据为一个批次
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     shuffle=True,#打乱顺序
                                     drop_last=True)#丢弃尾数

In [3]:
#全连接神经网络
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #定义神经网络结构
        self.fc = torch.nn.Sequential(#组合多层神经网络
            torch.nn.Linear(in_features=784, out_features=128),
            torch.nn.ReLU(),#激活函数
            torch.nn.Linear(in_features=128, out_features=256),
            torch.nn.ReLU(),#激活函数
            torch.nn.Linear(in_features=256, out_features=10),
            torch.nn.Softmax(dim=1)#归一化
        )
    #定义神经网络计算过程
    def forward(self, x):
        return self.fc(x)

In [4]:
#初始化神经网络
model = Model()

In [5]:
#初始化指标参数
TP=FP=TN=FN=overall=0
#定义指标计算函数
def count(x,y,target_label):
    global TP,FP,TN,FN,overall
    if x==y:
        overall+=1
    if x==target_label and y==target_label:
        TP+=1
    elif x==target_label and y!=target_label:
        FP+=1    
    elif x!=target_label and y!=target_label:
        TN+=1        
    else:
        FN+=1        
def printParameter():
    global TP,FP,TN,FN,overall
    overallAccuracy=overall/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
    accuracy=(TP+TN)/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
    precision=TP/(TP+FP)if (TP + FP) > 0 else 0
    recall=TP/(TP+FN)if (TP+FN) > 0 else 0
    F1=2*(precision*recall)/(precision+recall)if (precision+recall) > 0 else 0
    print(f"overall accuracy={overallAccuracy}")
    print(f"accuracy={accuracy}")
    print(f"precision={precision}")
    print(f"recall={recall}")
    print(f"F1={F1}")
def resetCount():
    global TP,FP,TN,FN,overall
    TP=TN=FP=FN=overall=0

In [6]:
#model = torch.load('fc_mnist.pth')

In [7]:
#定义训练过程
model.cuda()#在GPU上面训练
target_label=7#自定义正例，其他为负例
TP=TN=FP=FN=overall=0
def train():
    #优化器,根据梯度调整模型参数
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    #计算loss的函数
    loss_fun = torch.nn.CrossEntropyLoss()
    #让model进入train模式,开启dropout等功能
    model.train()
    bar=tqdm.tqdm(range(100))
    #全量数据遍历100轮
    for epoch in bar:
        for i, (x, y) in enumerate(loader):
            x=x.reshape(-1,784).cuda()
            y=y.cuda()
            output = model(x)
            loss = loss_fun(output, y)
            #根据loss计算模型的梯度
            loss.backward()
            #根据梯度调整模型的参数
            optimizer.step()
            #梯度归零,准备下一轮的计算
            optimizer.zero_grad()
        if epoch % 20 == 0:#每20轮训练打印一次模型精确度
            j=0
            for x in output.argmax(dim=1):
                count(x,y[j],target_label)
                j+=1
            print(f"epoch={epoch},loss={loss.item()/batch_size}")
            printParameter()
            resetCount()

In [8]:
#训练
train()

  1%|▊                                                                                 | 1/100 [00:01<02:46,  1.68s/it]

epoch=0,loss=0.054583415389060974
overall accuracy=0.84375
accuracy=0.96875
precision=1.0
recall=0.6666666666666666
F1=0.8


 21%|█████████████████                                                                | 21/100 [00:35<02:15,  1.71s/it]

epoch=20,loss=0.04801458865404129
overall accuracy=0.9375
accuracy=0.96875
precision=1.0
recall=0.6666666666666666
F1=0.8


 41%|█████████████████████████████████▏                                               | 41/100 [01:11<01:45,  1.79s/it]

...

 61%|█████████████████████████████████████████████████▍                               | 61/100 [01:45<01:11,  1.84s/it]

epoch=60,loss=0.04566667973995209
overall accuracy=1.0
accuracy=1.0
precision=1.0
recall=1.0
F1=1.0


 81%|█████████████████████████████████████████████████████████████████▌               | 81/100 [02:21<00:35,  1.89s/it]

epoch=80,loss=0.04567842185497284
overall accuracy=1.0
accuracy=1.0
precision=1.0
recall=1.0
F1=1.0


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:57<00:00,  1.78s/it]


In [9]:
#保存训练完的神经网络
torch.save(model, 'fc_mnist.pth')

In [10]:
#测试
#不计算模型梯度,节省计算资源
target_label=7#自定义正例，其他为负例
resetCount()
@torch.no_grad()
def test():
    #从磁盘加载模型
    model = torch.load('fc_mnist.pth')
    #模型进入测试模式,关闭dropout等功能
    model=model.eval().cuda()
    target_label=7#自定义正例，其他为负例
    TP=TN=FP=FN=overall=0
    #遍历整个数据集
    for i, (x, y) in enumerate(loader):
        x=x.reshape(-1,784).cuda()
        y=y.cuda()
        output = model(x)
        #求一个batch内各指标并累加
        j=0
        for x in output.argmax(dim=1):
            count(x,y[j],target_label)
            j+=1
    printParameter()#打印指标
    resetCount()#重置参数
test()

overall accuracy=0.9808693910256411
accuracy=0.9965945512820513
precision=0.9825072886297376
recall=0.9844206426484907
F1=0.9834630350194552
